In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")


llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002CB9B4A5E20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002CB9B6D3D90>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
	model_name="sentence-transformers/all-MiniLM-L6-v2",
	model_kwargs={"token": os.environ['HF_TOKEN']}
)

In [5]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatMessagePromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [7]:
#load chunk and index the contents

import bs4
loader = WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/Mahabharata",),
    bs_kwargs=dict(
        parse_only = bs4.SoupStrainer(
            class_=("firstHeading mw-first-heading", "mw-body-content")
        )
    ),
)

docs = loader.load()
docs

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Mahabharata'}, page_content='MahabharataMajor Indian epic\nThis article is about the Sanskrit epic. For other uses, see Mahabharata (disambiguation).\n\n\n\nMahabharataManuscript illustration of the Battle of KurukshetraInformationReligionHinduismAuthorVyasaLanguageSanskritPeriodPrincipally compiled in 3rd century BCE–4th century CEChapters18 ParvasVerses100,000Full textMahabharata at Sanskrit Wikisource Mahabharata at  English Wikisource\nPart of a series onHindu scriptures and texts\nShruti\nSmriti\nList\nVedas\nRigveda\nSamaveda\nYajurveda\nAtharvaveda\nDivisions\n\nSamhita\nBrahmana\nAranyaka\nUpanishads\n\nUpanishadsRig vedic\nAitareya\nKaushitaki\nSama vedic\n\nChandogya\nKena\nYajur vedic\n\nBrihadaranyaka\nIsha\nTaittiriya\nKatha\nShvetashvatara\nMaitri\nAtharva vedic\n\nMundaka\nMandukya\nPrashna\n\nOther scriptures\nAgamas\nBhagavad Gita\nTantras\n\nRelated Hindu texts\nVedangas\nShiksha\nChandas\nVyakarana\nNirukta

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)
retriever = vectorstore.as_retriever()
retriever


VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000002CBDB344760>, search_kwargs={})

In [10]:
##Prompt Template
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

system_prompt = (
"You are an assistant for questiong tasks,"
"Use the following pieces of retrieved context to answer"
"the question. if you dont know answer say thank you"
"dont know. use three sentences max and keep the"
"answer concise"
"\n\n"
"{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]

)
 

In [11]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [17]:
response = rag_chain.invoke({
    "input" : "what is the reason of war"
})

response['answer']

'The reason of war in the Mahabharata is a dynastic conflict between the Kauravas and the Pandavas, two branches of the same family, over the throne of Hastinapura.'

In [22]:
#Adding Chat History

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question"
    "which might refrence context in the chat history"
    "formulate a standalone question which can be understood"
    "without the chat history. DO NOT answer the question,"
    "just reformulate it if needed and otherwise return it as is"
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [26]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [27]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [50]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history = []
question = "what role did shri krishna played" 
response1 = rag_chain.invoke({"input":question, "chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"]) 
    ]
)

question2 = "when he first appeared"
response2=rag_chain.invoke({"input":question2, "chat_history":chat_history})
print("Answer 1:", response1['answer'])
print("Answer 2:", response2['answer'])


Answer 1: Shri Krishna played the role of an advisor and charioteer to Arjuna, reminding him of his duty as a Kshatriya to fight for a righteous cause in the Bhagavad Gita section of the epic.
Answer 2: Shri Krishna first appeared in the Mahabharata as a charioteer for Arjuna before the start of the Kurukshetra War.


In [51]:
chat_history

[HumanMessage(content='what role did shri krishna played', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Shri Krishna played the role of an advisor and charioteer to Arjuna, reminding him of his duty as a Kshatriya to fight for a righteous cause in the Bhagavad Gita section of the epic.', additional_kwargs={}, response_metadata={})]

In [52]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id: str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [53]:
conversational_rag_chain.invoke(
    {
        "input" : "What is cause of war"
    },
    config = {
        "configurable":{"session_id":"abc123"}
    },
)['answer']

'According to the Mahabharata, the cause of war is not specified as a single reason, but rather a complex and multifaceted issue. The text suggests that the war was sparked by a dynastic conflict between the Kauravas and the Pandavas, with each side having its own motivations and justifications for fighting.'